# 문서 검색 챗봇 만들기 LangChain

In [1]:
!pip install -q grobid-client langchain openai faiss-cpu PyPDF2 tiktoken

In [2]:
import openai
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

## Preprocess a PDF file

In [4]:
reader = PdfReader("./assets/qna-samples/ai_manual.pdf")

raw_text = ""

for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

raw_text[:1000]

'Q&A로 풀어본 \nAI 학습용 데이터\n상세 매뉴얼2019Q&A로 풀어본 \nAI 학습용 데이터\n상세 매뉴얼2019\nContents\n한국어-영어 번역 말뭉치 AI 데이터  2 1\n이상행동 CCTV 영상 AI 데이터  6 2\n한국어 글자체 이미지 AI 데이터  10 3\n인도 보행 영상 AI 데이터  14 4\n멀티모달 영상 AI 데이터  22 5\n사람 동작 영상 AI데이터  26 6\n한국인 안면 이미지 AI 데이터  30 7\n위해물품 엑스레이 이미지 AI 데이터  34 8\n질병진단 이미지 AI 데이터  38 9\n이상행동 CCTV 영상 AI 데이터  45 102\nQ&A로 풀어본 AI 학습용 데이터 상세 매뉴얼한국어-영어 번역 말뭉치 AI 데이터\n  세계는 이미 신경망 번역( NMT, Neural  Machine  Translation ) 엔진을 사용하는 것이 일반화되어, 용도에 맞는 제품과 \n서비스를 만들어 기존의 번역을 혁신하는 단계가 되었으나, 국내 자동번역/인공지능 번역 연구개발은 대규모 고품질의 \n말뭉치가 부족하여 개별 기업이 자체적 기술 개발에 한계\n  최근 기업에 신경망 번역 엔진을 공식적으로 도입하여 사용하는 사례가 증가하고 향후 업무 활용 및 연구개발이 더욱 \n활발해질 것으로 기대함에 따라 공공 번역 말뭉치를 구축하여 공개함 \n  신경망 번역 성능 향상을 위한 고품질 한국어-영어 번역 말뭉치 셋 160만 문장 구축; 뉴스 80만, 지자체 웹사이트 10만, \n조례 10만, 한국문화 10만, 구어체 40만, 대화체 10만1\n구축 공정필요성  \n구축 내용Q&A로 풀어본 AI 학습용 데이터 상세 매뉴얼\n구축 기관 (주)솔트룩스파트너스, (주)에버트란, (주)플리토\n수집 정제 번역 검수 배포3\nQ&A로 풀어본 AI 학습용 데이터 상세 매뉴얼수집된 뉴스데이터 종류는 무엇인가요?Q\nA대상매체는 국민일보, 내일신문, 노컷뉴스, 미디어오늘, 서울경제, 스포츠서울, 전자신문, 파이낸셜뉴스, \n한겨레, SBS의 10개 

## Summarize 요약

In [5]:
from langchain import OpenAI
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.summarize import load_summarize_chain

llm = OpenAI(temperature=0)
summary_chain = load_summarize_chain(llm, chain_type="map_reduce")

summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)

/Users/heewungsong/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAI instead.
  warn_deprecated(


In [6]:
summarize_document_chain.run(raw_text)

/Users/heewungsong/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' This document provides a detailed manual for AI training data, including various types of images and videos collected and refined by three companies. The data is valuable for developing AI translation engines and can be used for various purposes such as legal translation and social media. The manual also explains the process of collecting, refining, and verifying the data using annotation and ontology tools. It includes information on the dataset, image and annotation information, and licensing. The data is used for various applications such as autonomous driving and emotion analysis. The manual also discusses the development of an AI learning dataset for human motion recognition and emotion analysis. It includes a large database of facial images for facial recognition technology. The data set also includes X-ray images for airport and port security. The data is collected and processed with strict adherence to privacy laws and regulations. The report also discusses the need for AI da

## Question Answering 질문 답변

In [7]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo") # gpt-3.5-turbo, gpt-4

qa_chain = load_qa_chain(model, chain_type="map_reduce")
qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

/Users/heewungsong/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [8]:
qa_document_chain.run(
    input_document=raw_text,
    question="문서의 목차를 알려줘")

'죄송합니다. 해당 문서의 목차가 포함되어 있지 않습니다.'

In [9]:
qa_document_chain.run(
    input_document=raw_text,
    question="사물 이미지 데이터의 구축공정에 대해 설명해줘")

'사물 이미지 데이터의 구축 공정은 일반적으로 다음과 같은 단계로 이루어집니다:\n\n1. 데이터 수집: 사물 이미지 데이터를 수집하기 위해 다양한 방법을 사용합니다. 이는 인터넷에서 이미지를 크롤링하거나, 사진을 찍어서 데이터를 수집하는 것 등이 포함될 수 있습니다.\n\n2. 데이터 정제: 수집된 데이터는 정제 작업을 거쳐야 합니다. 이 단계에서는 중복되거나 품질이 낮은 이미지를 제거하고, 필요한 정보를 추출하여 데이터를 정리합니다.\n\n3. 어노테이션 작업: 이미지 내의 사물을 식별하고 분류하기 위해 어노테이션 작업을 수행합니다. 어노테이션은 주로 바운딩 박스, 세그멘테이션 마스크, 키포인트 등의 형태로 이루어집니다. 이 과정에서는 사물의 위치, 크기, 형태 등을 정확하게 표시하여 이미지에 대한 정보를 제공합니다.\n\n4. 데이터 포맷 변환: 어노테이션 작업이 완료된 데이터를 필요한 포맷으로 변환합니다. 이를 통해 다양한 딥러닝 프레임워크나 학습 알고리즘에서 사용할 수 있는 형태로 데이터를 구성합니다. 일반적으로 XML, JSON, CSV 등의 형식으로 데이터를 저장하고 관리합니다.\n\n5. 데이터 검수: 어노테이션 작업이 완료된 데이터를 검수하여 품질을 확인합니다. 이 과정에서 오검출, 미검출, 과검출 등의 문제를 확인하고 수정하는 작업이 이루어집니다. 검수를 통해 데이터의 정확성과 일관성을 유지하고 데이터셋의 품질을 관리합니다.\n\n6. 데이터 확장: 구축된 데이터셋을 확장하기 위해 필요한 추가 작업을 수행합니다. 이는 데이터 수집, 어노테이션 작업, 데이터 정제 등의 단계를 반복하여 데이터셋의 크기와 다양성을 향상시킵니다.\n\n7. 데이터 배포: 최종적으로 구축된 데이터는 공개 사이트나 라벨링 시스템을 통해 배포됩니다. 이를 통해 데이터를 수정하거나 추가 데이터를 업로드하여 다른 연구/개발자들이 활용할 수 있도록 합니다.\n\n이와 같은 과정을 거쳐 구축된 사물 이미지 데이터는 다양한 분야에서 컴퓨터 비전 및 인공지능 알고리즘의 학습과 평가

## Text Splitters

pdf 파일이 크기 때문에 적당한 크기로 split함

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:
reader = PdfReader("./assets/qna-samples/ai_manual.pdf")

raw_text = ""

for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

f"You have {len(raw_text)} texts"

'You have 40673 texts'

In [35]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)
documents = text_splitter.create_documents([raw_text])

In [36]:
f"You have {len(documents)} documents"

'You have 46 documents'

In [37]:
print(documents[0].page_content, "\n")
print(documents[1].page_content)

Q&A로 풀어본 
AI 학습용 데이터
상세 매뉴얼2019Q&A로 풀어본 
AI 학습용 데이터
상세 매뉴얼2019
Contents
한국어-영어 번역 말뭉치 AI 데이터  2 1
이상행동 CCTV 영상 AI 데이터  6 2
한국어 글자체 이미지 AI 데이터  10 3
인도 보행 영상 AI 데이터  14 4
멀티모달 영상 AI 데이터  22 5
사람 동작 영상 AI데이터  26 6
한국인 안면 이미지 AI 데이터  30 7
위해물품 엑스레이 이미지 AI 데이터  34 8
질병진단 이미지 AI 데이터  38 9
이상행동 CCTV 영상 AI 데이터  45 102
Q&A로 풀어본 AI 학습용 데이터 상세 매뉴얼한국어-영어 번역 말뭉치 AI 데이터
  세계는 이미 신경망 번역( NMT, Neural  Machine  Translation ) 엔진을 사용하는 것이 일반화되어, 용도에 맞는 제품과 
서비스를 만들어 기존의 번역을 혁신하는 단계가 되었으나, 국내 자동번역/인공지능 번역 연구개발은 대규모 고품질의 
말뭉치가 부족하여 개별 기업이 자체적 기술 개발에 한계
  최근 기업에 신경망 번역 엔진을 공식적으로 도입하여 사용하는 사례가 증가하고 향후 업무 활용 및 연구개발이 더욱 
활발해질 것으로 기대함에 따라 공공 번역 말뭉치를 구축하여 공개함 
  신경망 번역 성능 향상을 위한 고품질 한국어-영어 번역 말뭉치 셋 160만 문장 구축; 뉴스 80만, 지자체 웹사이트 10만, 
조례 10만, 한국문화 10만, 구어체 40만, 대화체 10만1
구축 공정필요성  
구축 내용Q&A로 풀어본 AI 학습용 데이터 상세 매뉴얼
구축 기관 (주)솔트룩스파트너스, (주)에버트란, (주)플리토
수집 정제 번역 검수 배포3
Q&A로 풀어본 AI 학습용 데이터 상세 매뉴얼수집된 뉴스데이터 종류는 무엇인가요?Q
A대상매체는 국민일보, 내일신문, 노컷뉴스, 미디어오늘, 서울경제, 스포츠서울, 전자신문, 파이낸셜뉴스, 

A대상매체는 국민일보, 내일신문, 노컷뉴스, 미디어오늘, 서울경제, 스포츠서울,

In [53]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

In [55]:
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents, embedding, persist_directory="db")

In [56]:
vectordb.persist()
vectordb = None

In [58]:
vectordb = Chroma(
    persist_directory="db", 
    embedding_function=embedding)

In [59]:
retriever = vectordb.as_retriever()

In [60]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x16b873c10>)

In [61]:
docs = retriever.get_relevant_documents("수집된 뉴스데이터의 종류는?")

In [62]:
docs

[Document(page_content='A대상매체는 국민일보, 내일신문, 노컷뉴스, 미디어오늘, 서울경제, 스포츠서울, 전자신문, 파이낸셜뉴스, \n한겨레, SBS의 10개 매체이며, 발행일이 2018년 6월 1일부터 2019년 5월 31일 사이의 뉴스데이터를 \n수집하였습니다.\n수집된 뉴스데이터 세부종류는 무엇인가요?Q\nA세부분류 항목은 문화, 경제, 정치, 스포츠, IT, 국제, 지역, 사회의 8개 항목입니다.\n원문 정제의 기준은 무엇인가요?Q\nA160만 문장은 평균 20어절 이상입니다. 그리고 뉴스의 경우 비슷한 기사가 많아서 85% 이상 중복되는 문장은 \n수집하지 않았습니다. 정제\n데이터 학습을 위한 번역 문장의 특성이 있을까요?Q\nA문장과 문장이 쌍을 이루어 딥러닝 학습을 효율적으로 할 수 있도록 사업 초기에 구축 가이드라인을 설정하고 \n작업방식을 통일해서 데이터를 구축했습니다. 문장부호 및 고유명사, 단위나 수치 표시 지침을 정확하게 세우고 \n준수하도록 했습니다. 번역수집된 뉴스데이터 종류는 무엇인가요?Q\nA과업 결과물에 포함된 디지털 뉴스의 이용 허락 기간은 『 2019 한국어-영어 번역 말뭉치 AI데이터 구축』 사업의 \n결과물로 공개되는 기간까지로 정했습니다. 본 사업이 공개 데이터로 유지되는 기간은 저작권의 이슈가 발생하지 \n않습니다 수집4\nQ&A로 풀어본 AI 학습용 데이터 상세 매뉴얼데이터의 구조나 메타정보를 알려주세요.Q\nA구축 데이터는 엑셀 파일(*. xlsx)로 제공하고 번역 DB를 다운받아 수월하게 활용 가능합니다. 데이터의 모든 \n문장에는 문장번호를 부착하여 관리가 용이합니다. 배포Q&A로 풀어본 AI 학습용 데이터 상세 매뉴얼\n구분 문장번호 출처 특성\n뉴스 ◯ 기사 URL 기사 작성일\n지자체 웹사이트 ◯\n조례 ◯ URL, 출판물\n구어체 ◯\n대화체 ◯ 대화세트/화자구분검수 시 번역 품질기준은 무엇인가요?Q\nA외부기관인 광운대학교 AI번역산업연구센터에서 본 결과물에 대한 품질검사를 진행했습니

In [63]:
print("\n\n".join([x.page_content[:200] for x in docs[:2]]))

A대상매체는 국민일보, 내일신문, 노컷뉴스, 미디어오늘, 서울경제, 스포츠서울, 전자신문, 파이낸셜뉴스, 
한겨레, SBS의 10개 매체이며, 발행일이 2018년 6월 1일부터 2019년 5월 31일 사이의 뉴스데이터를 
수집하였습니다.
수집된 뉴스데이터 세부종류는 무엇인가요?Q
A세부분류 항목은 문화, 경제, 정치, 스포츠, IT, 국제, 지역, 사회의 

A대상매체는 국민일보, 내일신문, 노컷뉴스, 미디어오늘, 서울경제, 스포츠서울, 전자신문, 파이낸셜뉴스, 
한겨레, SBS의 10개 매체이며, 발행일이 2018년 6월 1일부터 2019년 5월 31일 사이의 뉴스데이터를 
수집하였습니다.
수집된 뉴스데이터 세부종류는 무엇인가요?Q
A세부분류 항목은 문화, 경제, 정치, 스포츠, IT, 국제, 지역, 사회의 


In [64]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [67]:
from langchain.chains import RetrievalQA

In [68]:
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(), 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True)

In [70]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [74]:
query = "수집된 뉴스데이터의 종류는?"
llm_response = qa_chain(query)
llm_response["result"]

' 대상매체는 국민일보, 내일신문, 노컷뉴스, 미디어오늘, 서울경제, 스포츠서울, 전자신문, 파이낸셜뉴스이며, 발행일이 2018년 6월 1일부터 2019년 5월 31일 사이의 뉴스데이터를 수집하였습니다.'

' 대상매체는 국민일보, 내일신문, 노컷뉴스, 미디어오늘, 서울경제, 스포츠서울, 전자신문, 파이낸셜뉴스, 한겨레, SBS의 10개 매체이며, 발행일이 2018년 6월 1일부터 2019년 5월 31일 사이의 뉴스데이터를 수집하였습니다. 따라서 뉴스데이터가 수집된 종류는 총 10개이며, 문화, 경제, 정치, 스포츠, IT, 국제, 지역, 사회의 8개 항목으로 구분됩니다.'

In [75]:
query = "데이터를 활용할 수 있는 방법은?"
llm_response = qa_chain(query)
llm_response["result"]

' 본 데이터는 다양한 산업 분야에서 활용이 가능합니다. 예를 들어 스마트 교육, 스마트 관광, 스마트 공장/스토어 등에서 활용할 수 있습니다. 또한 OCR 기술을 활용한 자율주행, 증강현실, IoT 등의 산업분야에서도 활용이 가능합니다. 이미지를 인식하고 분류하는 기술을 개발하는데 사용할 수 있으며, 한글 글자체에 대한 학습용 데이터셋이 필요한 경우에도 사용할 수 있습니다. 따라서 데이터를 활용하는 방법은 다양하며, 적극적으로 활용하여 새로운 기술 및 서비스를 개발할 수 있습니다.'

In [77]:
query = "사물 이미지 데이터의 구축공정에서 데이터를 수집하는 방법은 무엇인가요?"
llm_response = qa_chain(query)
llm_response["result"]

' 데이터 수집은 직접 촬영을 통해 조달하고 나머지는 협회와 계약 또는 공공저작물로 사용권이 허락된 데이터를 수집하는 방식을 사용했습니다. 또한 웹크롤링을 통해 데이터를 수집하기도 합니다.'